# Nama: Stanly Sukmajaya
# Batch: RMT-036
## Tujuan dari notebook ini adalah untuk pengerjaan Live Code 2, dimana hal-hal yang diuji adalah pengetahuan SQL dan Pandas

In [120]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format
auth.authenticate_user()
print('Authenticated')

project_id_akun = "pc-api-5133159006662443041-958" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id_akun)

df = client.query('''
  SELECT *
  FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`;
''').to_dataframe()

df.head(10)

Authenticated


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_km,fare,tips,payment_type,company,pickup_location,dropoff_location
0,DP-250,D-7,2021-05-18 19:30:00+00:00,2021-05-18 19:45:00+00:00,3.06,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
1,DP-289,D-7,2021-05-30 15:15:00+00:00,2021-05-30 15:45:00+00:00,21.08,35.50,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027)
2,DP-316,D-7,2021-06-05 19:30:00+00:00,2021-06-05 19:45:00+00:00,7.24,14.00,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.8045320063 41.9850151008)
3,DP-354,D-7,2021-06-13 13:45:00+00:00,2021-06-13 14:15:00+00:00,22.37,37.25,0.00,Credit Card,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6559981815 41.9442266014)
4,DP-501,D-7,2021-07-07 15:15:00+00:00,2021-07-07 15:30:00+00:00,3.22,9.00,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6635175498 41.874005383)
5,DP-794,D-7,2021-08-25 13:45:00+00:00,2021-08-25 14:15:00+00:00,21.57,35.75,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6950125892 42.001571027)
6,DP-926,D-7,2021-09-21 20:30:00+00:00,2021-09-21 20:30:00+00:00,0.50,3.25,0.00,Cash,Deluc Porter,POINT (-87.9030396611 41.9790708201),POINT (-87.9030396611 41.9790708201)
7,DP-1470,D-7,2022-04-18 20:30:00+00:00,2022-04-18 20:30:00+00:00,0.50,3.25,0.00,Cash,Deluc Porter,POINT (-87.6558787862 41.96581197),POINT (-87.6558787862 41.96581197)
8,DP-1517,D-7,2022-04-28 22:30:00+00:00,2022-04-28 22:45:00+00:00,4.99,12.75,0.00,Cash,Deluc Porter,POINT (-87.6333080367 41.899602111),POINT (-87.6203346241 41.8571838585)
9,DP-1803,D-7,2022-07-14 18:45:00+00:00,2022-07-14 19:45:00+00:00,31.86,51.00,0.00,Cash,Deluc Porter,POINT (-87.913624596 41.9802643146),POINT (-87.6203346241 41.8571838585)


# Problem 1 - SQL

## 1. Apa saja nama perusahaan yang terdapat dalam data?
#### Terdapat 5 nama perusahaan yang unik, yaitu Deluc Porter, Synctaxi Inc., Atomic Inc., Federway Int. dan Rocket Cab Auto

In [127]:
query1 = '''
  SELECT DISTINCT company
  FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`;
''';

client.query(query1).to_dataframe()

,company
0,Deluc Porter
1,Synctaxi Inc.
2,Atomic Inc.
3,Federway Int.
4,Rocket Cab Auto


## 2. Berapa nilai minimum dan maksimum dari waktu pengambilan data?

#### Waktu minimum dan maksimum pengambilan data menunjukkan rentang 2 tahun, berarti dataset tersebut memiliki observasi data selama 2 tahun

In [126]:
query2 = '''
  SELECT
    MIN(trip_start_timestamp) AS min_waktu,
    MAX(trip_start_timestamp) AS max_waktu
  FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`;
''';

client.query(query2).to_dataframe()

,min_waktu,max_waktu
0,2021-01-02 11:00:00+00:00,2022-12-31 14:15:00+00:00


## 3. Bandingkan jumlah banyaknya perjalanan antara seluruh perusahaan pada tahun 2022?

#### Pada tahun 2022, perusahaan Synctaxi Inc. memiliki jumlah perjalanan terbanyak dibandingkan perusahaan lain, yang berarti kemungkinan besar perusahaan ini paling banyak di pakai pelanggan, sedangkan Atomic Inc. memiliki jumlah perjalanan paling sedikit, yaitu 0. Kemungkinan Atomic Inc. sudah berhenti beroperasi di tahun 2022

In [130]:
query3 = '''
  SELECT DISTINCT h.company, IFNULL(subquery.total_trip, 0) AS total_trip
  FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare` h
    LEFT JOIN (
      SELECT company,
      COUNT(unique_key) AS total_trip,
      EXTRACT(YEAR FROM trip_start_timestamp) AS year
      FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`
      GROUP BY company, year
      HAVING year = 2022
      ORDER BY IFNULL(total_trip, 0) DESC
    ) AS subquery
  ON h.company = subquery.company
  ORDER BY total_trip DESC
''';

client.query(query3).to_dataframe()

,company,total_trip
0,Synctaxi Inc.,1341
1,Federway Int.,1059
2,Deluc Porter,1050
3,Rocket Cab Auto,77
4,Atomic Inc.,0


## 4. Perusahaan mana yang memiliki tarif perjalanan per km termurah?

### Rocket Cab Auto memiliki tarif perjalanan per km termurah, dengan nilai 2.49

In [141]:
query4 = '''
  SELECT
    company,
    AVG(fare / trip_km) AS average_fare_per_km
  FROM `ftds-hacktiv8-project.phase0_ftds_036_rmt.taxi_fare`
  GROUP BY company
  ORDER BY average_fare_per_km;
''';

client.query(query4).to_dataframe()

,company,average_fare_per_km
0,Rocket Cab Auto,2.49
1,Deluc Porter,3.83
2,Atomic Inc.,3.86
3,Synctaxi Inc.,3.95
4,Federway Int.,4.26


# Problem 2 - Pandas

## 1. Buat kolom baru, dengan nama Keterangan_Waktu. Kolom ini berisikan label yang menunjukkan representasi keterangan waktu berdasarkan jam dengan ketentuan berikut:
#### Keterangan Waktu	Jam
#### Pagi	04:00 - 09:59
#### Siang	10:00 - 15:59
#### Sore	16:00 - 18:59
#### Malam	19:00 - 23:59
#### Dini Hari	00:00 - 03:59

In [123]:
for index, row in df.iterrows():
  the_time = row['trip_start_timestamp'].strftime('%H:%M')
  if the_time >= '04:00' and the_time <= '09:59':
    df.at[index, 'Keterangan_Waktu'] = 'Pagi'
  elif the_time >= '10:00' and the_time <= '15:59':
    df.at[index, 'Keterangan_Waktu'] = 'Siang'
  elif the_time >= '16:00' and the_time <= '18:59':
    df.at[index, 'Keterangan_Waktu'] = 'Sore'
  elif the_time >= '19:00' and the_time <= '23:59':
    df.at[index, 'Keterangan_Waktu'] = 'Malam'
  else:
    df.at[index, 'Keterangan_Waktu'] = 'Dini Hari'
print(df[["trip_start_timestamp", "Keterangan_Waktu"]].head(25))

        trip_start_timestamp Keterangan_Waktu
0  2021-05-18 19:30:00+00:00            Malam
1  2021-05-30 15:15:00+00:00            Siang
2  2021-06-05 19:30:00+00:00            Malam
3  2021-06-13 13:45:00+00:00            Siang
4  2021-07-07 15:15:00+00:00            Siang
5  2021-08-25 13:45:00+00:00            Siang
6  2021-09-21 20:30:00+00:00            Malam
7  2022-04-18 20:30:00+00:00            Malam
8  2022-04-28 22:30:00+00:00            Malam
9  2022-07-14 18:45:00+00:00             Sore
10 2022-07-17 19:15:00+00:00            Malam
11 2021-03-27 14:00:00+00:00            Siang
12 2021-06-26 16:30:00+00:00             Sore
13 2021-07-08 14:30:00+00:00            Siang
14 2021-07-31 21:30:00+00:00            Malam
15 2021-08-21 16:30:00+00:00             Sore
16 2021-11-12 21:00:00+00:00            Malam
17 2022-01-03 09:00:00+00:00             Pagi
18 2022-02-12 20:15:00+00:00            Malam
19 2022-03-05 11:15:00+00:00            Siang
20 2022-03-14 13:45:00+00:00      

## 2. Berapa total jarak perjalanan yang dicapai perusahaan Federway Int. pada tahun 2022?

### Perusahaan Federway Int. memiliki total jarak perjalanan 4359.24 km pada tahun 2022

In [122]:
df[(df["company"] == "Federway Int.") & (df["trip_start_timestamp"].dt.year == 2022)]["trip_km"].sum()

4359.24135

### Perbandingan antar perusahaan pada tahun 2022 perihal total jarak perjalanan

In [121]:
df["company"] = df["company"].astype("category")
df[df["trip_start_timestamp"].dt.year == 2022].groupby("company")["trip_km"].sum().sort_values(ascending=False)

<ipython-input-121-fca3e205614b>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[df["trip_start_timestamp"].dt.year == 2022].groupby("company")["trip_km"].sum().sort_values(ascending=False)


,trip_km
company,
Deluc Porter,"18,515.22"
Synctaxi Inc.,"18,395.66"
Federway Int.,"4,359.24"
Rocket Cab Auto,"1,031.10"
Atomic Inc.,0.00
